In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import string
import matplotlib.pyplot as plt
import seaborn as sns
import re

from collections import Counter
from sklearn import ensemble
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
from sklearn.cluster import MiniBatchKMeans

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('C://duh/Thinkful/Thinkful/Exercises/Medium_AggregatedData.csv')
df.drop(['audioVersionDurationSec', 'codeBlock', 'codeBlockCount', 'collectionId', 'imageCount', 'isSubscriptionLocked', 'linksCount', 'postId', 'readingTime', 'responsesCreatedCount', 'socialRecommendsCount', 'tagsCount', 'publicationfacebookPageName', 'publicationfollowerCount', 'publicationpublicEmail', 'publicationdomain', 'publicationslug', 'publicationtwitterUsername', 'userId', 'userName', 'usersFollowedByCount', 'usersFollowedCount'], 1, inplace=True)
df.drop(['createdDatetime', 'latestPublishedDate', 'vote', 'scrappedDate', 'latestPublishedDatetime', 'slug', 'name', 'postCount', 'bio', 'firstPublishedDatetime', 'uniqueSlug', 'updatedDate', 'updatedDatetime'], 1, inplace=True)
df.drop(df.index[df.language != 'en'], inplace=True)
df = df[:int(len(df)/100)]
df = df.reset_index(drop=True)
nlp = spacy.load('en')
df['doc'] = None
df['sents'] = None
df['bow'] = None

dfsents = pd.DataFrame()

In [3]:
for x in range(len(df.text)):
    if x % 100 == 0:
        print("    Processing row {}".format(x))
    df.text[x] = re.sub(df.title[x], '', df.text[x])
    df.text[x] = df.text[x].lower()
    df.text[x] = re.sub('https://*.*?\\n', ' ', df.text[x])
    df.text[x] = re.sub('http://*.*?\\n', ' ', df.text[x])
    df.text[x] = re.sub('\\n', ' ', df.text[x])
    df.text[x] = re.sub('bibliography.*', '', df.text[x])
    df.text[x] = re.sub('[%s]' % re.escape(string.punctuation), '', df.text[x])
    df.text[x] = re.sub('\w*\d\w*', '', df.text[x])
    df.text[x] = re.sub('’', '', df.text[x])
    df.text[x] = re.sub('–', ' ', df.text[x])
    df.text[x] = df.text[x].strip()
    if df.text[x] == '':
        df.text[x] = 'and'
    

    Processing row 0
    Processing row 100
    Processing row 200
    Processing row 300
    Processing row 400
    Processing row 500
    Processing row 600
    Processing row 700
    Processing row 800
    Processing row 900
    Processing row 1000
    Processing row 1100
    Processing row 1200
    Processing row 1300
    Processing row 1400
    Processing row 1500
    Processing row 1600
    Processing row 1700
    Processing row 1800
    Processing row 1900
    Processing row 2000
    Processing row 2100
    Processing row 2200
    Processing row 2300
    Processing row 2400
    Processing row 2500


In [4]:
for x in range(len(df.text)):
    df.doc[x] = nlp(df.text[x])
    sents = [[sent, df.author[x]] for sent in df.doc[x].sents]
    dfsents = dfsents.append(sents)
    dfsents = dfsents.reset_index(drop=True)
    
sentences = dfsents.reset_index(drop=True)

In [5]:
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence'][::100]):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 50 == 0:
            print("Processing row {}".format(i))
            
    return df

In [6]:
for x in range(len(df.text)):
    df.bow[x] = bag_of_words(df.doc[x])

In [7]:
common_words = set()
for x in range(len(df.text)):
    common_words.update(df.bow[x])

In [ ]:
word_counts = bow_features(sentences, common_words)

Processing row 0


In [ ]:
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

In [ ]:
X_norm = normalize(X)

X_pca = PCA(.95).fit_transform(X_norm)

# Calculate predicted values.
y_pred = KMeans(random_state=0).fit_predict(X_pca)

In [ ]:
plt.figure(figsize=(20,10))
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y_pred)
plt.scatter(X_pca[:, 2], X_pca[:, 3], c=y_pred)
plt.scatter(X_pca[:, 4], X_pca[:, 5], c=y_pred)
plt.scatter(X_pca[:, 6], X_pca[:, 7], c=y_pred)
plt.scatter(X_pca[:, 8], X_pca[:, 9], c=y_pred)
plt.scatter(X_pca[:, 10], X_pca[:, 11], c=y_pred)
plt.scatter(X_pca[:, 12], X_pca[:, 13], c=y_pred)
plt.scatter(X_pca[:, 14], X_pca[:, 15], c=y_pred)
plt.scatter(X_pca[:, 16], X_pca[:, 17], c=y_pred)
plt.scatter(X_pca[:, 18], X_pca[:, 19], c=y_pred)
plt.scatter(X_pca[:, 20], X_pca[:, 21], c=y_pred)
plt.scatter(X_pca[:, 22], X_pca[:, 23], c=y_pred)
plt.scatter(X_pca[:, 24], X_pca[:, 25], c=y_pred)
plt.scatter(X_pca[:, 26], X_pca[:, 27], c=y_pred)
plt.scatter(X_pca[:, 28], X_pca[:, 29], c=y_pred)
plt.scatter(X_pca[:, 30], X_pca[:, 31], c=y_pred)
plt.scatter(X_pca[:, 32], X_pca[:, 33], c=y_pred)
plt.scatter(X_pca[:, 34], X_pca[:, 35], c=y_pred)

plt.show()

In [ ]:
lr = LogisticRegression()
train = lr.fit(X_train, y_train)
        
print('Training score: ', lr.score(X_train, y_train))
print('Test score: ', lr.score(X_test, y_test))

In [ ]:
svm = SVC(kernel='linear')
train = svm.fit(X_train, y_train)

print('Training score: ', svm.score(X_train, y_train))
print('Test score: ', svm.score(X_test, y_test))

In [ ]:
X_pca.shape

In [4]:
df

,createdDate,firstPublishedDate,language,recommends,subTitle,text,title,totalClapCount,url,wordCount,publicationdescription,publicationname,publicationtags,tag_name,author,doc,sents,bow
0,2018-09-18,2018-09-18,en,2,A major private IT company implements blockcha...,a major private it company implements blockcha...,"Private Business, Government and Blockchain",100,https://medium.com/s/story/private-business-go...,201,ICOBox is the first and the biggest new genera...,ICOBox,"BLOCKCHAIN,ICO,ETHEREUM,ETHEREUM BLOCKCHAIN,TO...",Blockchain,Anar Babaev,None,None,None
1,2018-09-18,2018-09-18,en,2,A major private IT company implements blockcha...,a major private it company implements blockcha...,"Private Business, Government and Blockchain",100,https://medium.com/s/story/private-business-go...,201,ICOBox is the first and the biggest new genera...,ICOBox,"BLOCKCHAIN,ICO,ETHEREUM,ETHEREUM BLOCKCHAIN,TO...",Samsung,Anar Babaev,None,None,None
2,2018-09-18,2018-09-18,en,2,A major private IT company implements blockcha...,a major private it company implements blockcha...,"Private Business, Government and Blockchain",100,https://medium.com/s/story/private-business-go...,201,ICOBox is the first and the biggest new genera...,ICOBox,"BLOCKCHAIN,ICO,ETHEREUM,ETHEREUM BLOCKCHAIN,TO...",It,Anar Babaev,None,None,None
3,2018-01-07,2018-01-07,en,0,Introduction,epq draft words introduction automation is ...,EPQ draft 1 (4844 words),0,https://medium.com/s/story/introduction-3-1000...,4854,NaN,NaN,NaN,Technology,George Sykes,None,None,None
4,2018-01-07,2018-01-07,en,0,Introduction,epq draft words introduction automation is ...,EPQ draft 1 (4844 words),0,https://medium.com/s/story/introduction-3-1000...,4854,NaN,NaN,NaN,Robotics,George Sykes,None,None,None
5,2018-01-07,2018-01-07,en,0,Introduction,epq draft words introduction automation is ...,EPQ draft 1 (4844 words),0,https://medium.com/s/story/introduction-3-1000...,4854,NaN,NaN,NaN,Automation,George Sykes,None,None,None
6,2018-01-07,2018-01-07,en,0,Introduction,epq draft words introduction automation is ...,EPQ draft 1 (4844 words),0,https://medium.com/s/story/introduction-3-1000...,4854,NaN,NaN,NaN,Epq,George Sykes,None,None,None
7,2018-03-08,2018-03-08,en,0,Various associations in the present days are o...,various associations in the present days are o...,"Ascent of data Science, SAS and Big data Analy...",0,https://medium.com/s/story/ascent-of-data-scie...,533,NaN,NaN,NaN,Data Science,tech data,None,None,None
8,2018-03-08,2018-03-08,en,0,Various associations in the present days are o...,various associations in the present days are o...,"Ascent of data Science, SAS and Big data Analy...",0,https://medium.com/s/story/ascent-of-data-scie...,533,NaN,NaN,NaN,Sas Training In Pune,tech data,None,None,None
9,2018-03-08,2018-03-08,en,0,Various associations in the present days are o...,various associations in the present days are o...,"Ascent of data Science, SAS and Big data Analy...",0,https://medium.com/s/story/ascent-of-data-scie...,533,NaN,NaN,NaN,Hadoop Training Mumbai,tech data,None,None,None
